## Notebook to convert the 'average' input file to a new parameter file iteration

##### In this case to update the default pfile with the set of adjested parameter values from the averaged output of the SP calibraiton

In [1]:
# Find the parameters that are adjusted in the calibration process

In [2]:
import xarray as xr
import os
import netCDF4 as nc
import matplotlib.pyplot as plt
import shutil
import numpy as np

### Control vars

In [3]:
tag_param_file='fates_params_vertsc_dayl.nc'
newpfilename='fates_params_vertsc_dayl_SPcal_4.nc'
pfilesdir='../intermediate_pfiles/'
averagefile = 'average4.nc'

### Find the variables that are calibrated in the SP calibration procedure

##### Compare two files in the SP LhC output

In [4]:

directory_path = '..//lhc_param_files'

file='FATES_LH_347.nc'
file_path = os.path.join(directory_path, file)
data1 = xr.open_dataset(file_path)
file='FATES_LH_348.nc'
file_path = os.path.join(directory_path, file)
data2 = xr.open_dataset(file_path)

common_variables = set(data1.variables) & set(data2.variables)

# Iterate through the common variables and compare their values
parlist=[]
for var_name in common_variables:
    var_data1 = data1[var_name]
    var_data2 = data2[var_name]
    # Compare the values of the variables
    if not var_data1.equals(var_data2):
        
        print(f"Values of variable '{var_name}' are different")
        parlist.append(var_name)
print(parlist)

Values of variable 'fates_leaf_vcmaxse' are different
Values of variable 'fates_leaf_stomatal_intercept' are different
Values of variable 'fates_maintresp_leaf_atkin2017_baserate' are different
Values of variable 'fates_leaf_vcmaxha' are different
Values of variable 'fates_leaf_jmaxse' are different
Values of variable 'fates_nonhydro_smpsc' are different
Values of variable 'fates_leaf_vcmax25top' are different
Values of variable 'fates_leaf_vcmaxhd' are different
Values of variable 'fates_leaf_stomatal_slope_ballberry' are different
['fates_leaf_vcmaxse', 'fates_leaf_stomatal_intercept', 'fates_maintresp_leaf_atkin2017_baserate', 'fates_leaf_vcmaxha', 'fates_leaf_jmaxse', 'fates_nonhydro_smpsc', 'fates_leaf_vcmax25top', 'fates_leaf_vcmaxhd', 'fates_leaf_stomatal_slope_ballberry']


#### Create and read in the new base parameter file
- Use the default parameter file from the new FATES tag
- and make a new version of this, 
- and then copy the variables from the averaged file into that.
- remembering that the number of PFTs is different


In [5]:
newf=pfilesdir+newpfilename
tagfile=pfilesdir+tag_param_file
print('tag file',tagfile)
print('newf',newf)


tag file ../intermediate_pfiles/fates_params_vertsc_dayl.nc
newf ../intermediate_pfiles/fates_params_vertsc_dayl_SPcal_4.nc


In [6]:
tagfiledata = xr.open_dataset(tagfile)
print(tagfiledata.fates_leaf_vcmax25top.values)
print(tagfiledata.fates_frag_maxdecomp.values)

[[50. 62. 39. 61. 58. 58. 62. 54. 54. 78. 78. 78.]]
[5.20e-01 3.83e-01 3.83e-01 1.90e-01 1.00e+00 9.99e+02]


#### Read in the SP calibrated average parameter file

In [7]:
averagepfile = xr.open_dataset(pfilesdir+averagefile)
print(averagepfile.fates_leaf_vcmax25top.values)
print(tagfiledata.fates_frag_maxdecomp.values)

[[29.9373648  33.2337325  47.38454974 61.         58.         58.
  62.         54.         54.         45.89753118 23.1667347  49.42265484
  78.         78.        ]]
[5.20e-01 3.83e-01 3.83e-01 1.90e-01 1.00e+00 9.99e+02]


#### Copy the value of the parameters in the average file into the new baseline file

In [8]:
tagfile_data = xr.open_dataset(tagfile)

for var_name in parlist:
    print(var_name)
    base_var = tagfile_data[var_name].values
    average_var = averagepfile[var_name].values
    print('base',base_var[0:3])
    print('av',average_var[0:3])
    if tagfile_data[var_name].ndim == 1:
        base_var[0:12] = average_var[0:12]
    else:
        base_var[:, 0:12] = average_var[:, 0:12]

    tagfile_data[var_name].values = base_var  # Update the variable values in the dataset

tagfile_data.to_netcdf(newf)  # Save the modified dataset back to the original file
tagfile_data.close()

fates_leaf_vcmaxse
base [485. 485. 485.]
av [559.23468856 440.87696678 469.25607086]
fates_leaf_stomatal_intercept
base [10000. 10000. 10000.]
av [  5666.85391194  33315.31798552 111020.66103126]
fates_maintresp_leaf_atkin2017_baserate
base [1.756  1.4995 1.4995]
av [2.0306485  2.05484886 1.880617  ]
fates_leaf_vcmaxha
base [65330. 65330. 65330.]
av [62358.31817956 69140.46772146 66486.2536002 ]
fates_leaf_jmaxse
base [495. 495. 495.]
av [544.98760814 437.10191078 465.4758702 ]
fates_nonhydro_smpsc
base [-255000. -255000. -255000.]
av [-261482.70388932 -255284.76314456 -251193.89963948]
fates_leaf_vcmax25top
base [[50. 62. 39. 61. 58. 58. 62. 54. 54. 78. 78. 78.]]
av [[29.9373648  33.2337325  47.38454974 61.         58.         58.
  62.         54.         54.         45.89753118 23.1667347  49.42265484
  78.         78.        ]]
fates_leaf_vcmaxhd
base [149250. 149250. 149250.]
av [145851.77931398 161182.69826226 150804.5994506 ]
fates_leaf_stomatal_slope_ballberry
base [8. 8. 8.]
a

In [9]:
newpfile=xr.open_dataset(newf)
print(newf)
print(newpfile['fates_leaf_vcmax25top'].values)
print(newpfile['fates_maintresp_leaf_atkin2017_baserate'].values)
print(newpfile['fates_alloc_organ_id'].values)

../intermediate_pfiles/fates_params_vertsc_dayl_SPcal_4.nc
[[29.9373648  33.2337325  47.38454974 61.         58.         58.
  62.         54.         54.         45.89753118 23.1667347  49.42265484]]
[2.0306485  2.05484886 1.880617   1.756      1.756      1.756
 2.0749     2.0749     2.0749     1.93625946 2.00627512 1.94858872]
[1. 2. 3. 6.]


In [14]:
tagfile_data = xr.open_dataset(tagfile)
for var_name in newpfile.variables:

    if var_name in averagepfile.variables:
        new_var = newpfile.variables[var_name]
        tag_var = tagfile_data.variables[var_name]
        #print(var_name,'ndim',average_var.ndim)
        #print(var_name,new_var.ndim)
        #if(var_name,np.shape(new_var) == np.shape(average_var)
        if(new_var.ndim==1):
            new_var=new_var[:12]
            tag_var=tag_var[:12]
            diffs = abs(new_var-tag_var)
            sd=np.sum(diffs.values[:])
        elif new_var.ndim==0:
            diffs = abs(new_var-tag_var)
            sd=np.sum(diffs.values)

        elif new_var.ndim==2 and var_name != 'fates_hlm_pft_map':
            new_var=new_var[:,0:12]
            tag_var=tag_var[:,0:12]
            print('2d shape',var_name,np.shape(new_var),np.shape(tag_var))
            diffs = abs(new_var-tag_var)
            sd=np.sum(diffs.values[:])

        if(var_name in parlist):
            print('SP var',var_name)
            print('tag',tag_var.values[0:4])
            print('new',new_var.values[0:4])
                  
        if(np.dtype(new_var) =='float64'):            
            if (sd>0.001):
                
                print(f"Variable {var_name} is different in basepfile and averagepfile")
               # print('av',average_var.values[0])
               # print('base',base_var.values[0])
                #newpfile.variables[var_name]=base_var
        else:
            print(f"Variable {var_name} is in basepfile but not in averagepfile")


2d shape fates_alloc_organ_priority (4, 12) (4, 12)
2d shape fates_cnp_turnover_nitr_retrans (4, 12) (4, 12)
2d shape fates_cnp_turnover_phos_retrans (4, 12) (4, 12)
2d shape fates_hydro_avuln_node (4, 12) (4, 12)
2d shape fates_hydro_epsil_node (4, 12) (4, 12)
2d shape fates_hydro_fcap_node (4, 12) (4, 12)
2d shape fates_hydro_kmax_node (4, 12) (4, 12)
2d shape fates_hydro_p50_node (4, 12) (4, 12)
2d shape fates_hydro_pinot_node (4, 12) (4, 12)
2d shape fates_hydro_pitlp_node (4, 12) (4, 12)
2d shape fates_hydro_resid_node (4, 12) (4, 12)
2d shape fates_hydro_thetas_node (4, 12) (4, 12)
2d shape fates_hydro_vg_alpha_node (4, 12) (4, 12)
2d shape fates_hydro_vg_m_node (4, 12) (4, 12)
2d shape fates_hydro_vg_n_node (4, 12) (4, 12)
SP var fates_leaf_jmaxse
tag [495. 495. 495. 495.]
new [544.98760814 437.10191078 465.4758702  495.        ]
Variable fates_leaf_jmaxse is different in basepfile and averagepfile
SP var fates_leaf_stomatal_intercept
tag [10000. 10000. 10000. 10000.]
new [  566